In [95]:
# Import the FPDF library
from fpdf import FPDF

In [111]:
import yfinance as yf

from key_ratios_and_financial_metrics import (extract_financial_data, pe_ratio,pb_ratio, ps_ratio, ev_ebitda, ev_to_revenue, gross_margin, operating_margin, net_margin, roa, asset_turnover, dividend_yield, earnings_yield, price_to_cash_flow)

ticker = yf.Ticker("RWC.AX")
data = extract_financial_data(ticker)

In [ ]:
pdf = FPDF()
# page 1
pdf.add_page()

# Define hex to RGB conversion function to allow coloured text
def hex_to_rgb(h):
    h = h.lstrip('#')
    return tuple(int(h[i:i+2], 16) for i in (0, 2, 4))

# Here, we will set out the first page of te PDF by creating multiple cells and adding the necessary text to them 
# Add a cell for the company logo 
pdf.image('RWC.png', 11, 10, 40, 10)

# Add a cell next to the multi_cell for report name and date 
# add first line 
pdf.set_xy(125, 10)
pdf.set_font('Arial', 'B', 10)
hex_col = '02B8E6'
r, g, b = hex_to_rgb(hex_col)
pdf.set_text_color(r, g, b)
pdf.cell(0, 5, f'FINM3422 Equity Research Report', ln=2)
# normal second line
pdf.set_font('Arial', '', 10)
pdf.set_text_color(0, 0, 0)
pdf.cell(0, 5, '30th April 2025', ln=1)

# add a cell for the report title 
pdf.set_xy(10, 30)
pdf.set_font('Arial', 'B', 16)
hex_col = '02B8E6'
r, g, b = hex_to_rgb(hex_col)
pdf.set_text_color(r, g, b)
pdf.cell(0, 5, f'Reliance Worldwide Corporation Ltd', ln=2)

# add a cell to the right of the report title for valuation status
pdf.set_xy(125, 30)
pdf.set_font('Arial', 'B', 14)
pdf.set_text_color(0, 0, 0)
pdf.cell(0, 5, f'Underweight', ln=2)

# Add a multi_cell with text that will be wrapped
pdf.set_xy(10, 37)
pdf.set_font('Arial', '', 12)
pdf.set_text_color(0, 0, 0)
pdf.multi_cell(110, 6, 'Reliance Holds the Line on Plumbing Strength as Valuation Tightens and Cyclical Risks Mount ', align='L')

# Add a cell to the right of the multi_cell for company details 
pdf.set_xy(125, 37)
pdf.set_font('Arial', 'B', 10)
pdf.set_text_color(0, 0, 0)
pdf.cell(0, 5, f'ASX: RWC', ln=2)
pdf.set_font('Arial', '', 10)
pdf.cell(0, 5, f'Current Price: $3.96', ln=2)
pdf.cell(0, 5, f'Target Price: $3.24', ln=1)
# Add a border under the cell
pdf.set_xy(10, 50)
pdf.set_line_width(0.5)
pdf.set_draw_color(0, 0, 0)
pdf.line(125, 53, 200, 53)

# Add a multi_cell with text that will be wrapped and import the text from the company overview txt file
pdf.set_xy(10, 55)
pdf.set_font('Arial', 'B', 11)
pdf.set_text_color(0, 0, 0)
pdf.cell(0, 5, f'Company Overview', ln=2)
pdf.set_font('Arial', '', 10)
pdf.set_text_color(0, 0, 0)
with open('Overview_1.txt', 'r') as file:
    text = file.read()
pdf.multi_cell(110, 6, text, align='J')

# Add a cell to the right of the multi_cell for author details 
pdf.set_xy(125, 55)
pdf.set_font('Arial', 'B', 10)
pdf.set_text_color(0, 0, 0)
pdf.cell(0, 5, f'Tom Scriven', ln=2)
pdf.set_font('Arial', '', 10)
pdf.cell(0, 5, f'47431681', ln=2)
pdf.cell(0, 5, f't.scriven@uqconnect.edu.au', ln=2)
pdf.set_font('Arial', 'B', 10)
pdf.set_text_color(0, 0, 0)
pdf.cell(0, 5, f'Felix Cavalerie', ln=2)
pdf.set_font('Arial', '', 10)
pdf.cell(0, 5, f'48032722', ln=2)
pdf.cell(0, 5, f'f.cavalerie@uqconnect.edu.au', ln=2)
pdf.set_font('Arial', 'B', 10)
pdf.set_text_color(0, 0, 0)
pdf.cell(0, 5, f'Edward Boniface', ln=2)
pdf.set_font('Arial', '', 10)
pdf.cell(0, 5, f'48825924', ln=2)
pdf.cell(0, 5, f'e.boniface@uqconnect.edu.au', ln=2)
pdf.set_font('Arial', 'B', 10)
pdf.set_text_color(0, 0, 0)
pdf.cell(0, 5, f'Ronit Relan', ln=2)
pdf.set_font('Arial', '', 10)
pdf.cell(0, 5, f'48823854', ln=2)
pdf.cell(0, 5, f'r.relan@uqconnect.edu.au', ln=2)
# add a border under the cell
pdf.set_xy(10, 115)
pdf.set_line_width(0.5)
pdf.set_draw_color(0, 0, 0)
pdf.line(125, 117, 200, 117)

# Under the company overview add a multi cell with text that will be wrapped and import the text from the finance txt file
pdf.set_xy(10, 160)
pdf.set_font('Arial', 'B', 11)
pdf.set_text_color(0, 0, 0)
pdf.cell(0, 5, f'Financial Performance', ln=2)
pdf.set_font('Arial', '', 10)
pdf.set_text_color(0, 0, 0)
with open('Finance_2.txt', 'r') as file:
    text = file.read()
pdf.multi_cell(110, 6, text, align='J')

# Under the author details add a cell for the historical share price chart
pdf.set_xy(125, 119)
pdf.set_font('Arial', 'B', 10)
pdf.set_text_color(0, 0, 0)
pdf.cell(0, 5, f'Historical Share Price', ln=2)

# Add a line under the cell
pdf.set_xy(125, 175)
pdf.set_line_width(0.5)
pdf.set_draw_color(0, 0, 0)
pdf.line(125, 175, 200, 175)

# Under the historical share price add a cell for the financial analysis section 
pdf.set_xy(125, 177)
pdf.set_font('Arial', 'B', 10)
pdf.set_text_color(0, 0, 0)
pdf.cell(0, 5, f'Valuation Ratios', ln=2)
pdf.set_font('Arial', '', 10)
pdf.cell(0, 5, f'P/E Ratio: {pe_ratio(data["price"], data["eps"]):.2f}', ln=2)
pdf.cell(0, 5, f'P/B Ratio: {pb_ratio(data["price"], data["book_value"]):.2f}', ln=2)
pdf.cell(0, 5, f'P/S Ratio: {ps_ratio(data["price"], data["revenue"]):.2f}', ln=2)
pdf.cell(0, 5, f'EV/EBITDA: {ev_ebitda(data["enterprise_value"], data["ebitda"]):.2f}', ln=2)
pdf.cell(0, 5, f'EV/Revenue: {ev_to_revenue(data["enterprise_value"], data["revenue"]):.2f}', ln=2)

pdf.set_xy(125, 208)
pdf.set_font('Arial', 'B', 10)
pdf.set_text_color(0, 0, 0)
pdf.cell(0, 5, f'Profitability Ratios', ln=2)
pdf.set_font('Arial', '', 10)
pdf.cell(0, 5, f'Gross Margin: {gross_margin(data["gross_profit"], data["revenue"]):.2f}', ln=2)
pdf.cell(0, 5, f'Operating Margin: {operating_margin(data["operating_income"], data["revenue"]):.2f}', ln=2)
pdf.cell(0, 5, f'Net Margin: {net_margin(data["net_income"], data["revenue"]):.2f}', ln=2)
pdf.cell(0, 5, f'ROA: {roa(data["net_income"], data["total_assets"]):.2f}', ln=2)

pdf.set_xy(125, 235)
pdf.set_font('Arial', 'B', 10)
pdf.set_text_color(0, 0, 0)
pdf.cell(0, 5, f'Efficiency Ratios', ln=2)
pdf.set_font('Arial', '', 10)
pdf.cell(0, 5, f'Asset Turnover: {asset_turnover(data["revenue"], data["total_assets"]):.2f}', ln=2)

pdf.set_xy(125, 247)
pdf.set_font('Arial', 'B', 10)
pdf.set_text_color(0, 0, 0)
pdf.cell(0, 5, f'Market Metrics', ln=2)
pdf.set_font('Arial', '', 10)
pdf.cell(0, 5, f'Dividend Yield: {dividend_yield(data["dividend"], data["price"]):.2f}', ln=2)
pdf.cell(0, 5, f'Earnings Yield: {earnings_yield(data["eps"], data["price"]):.2f}', ln=2)
pdf.cell(0, 5, f'Price to Cash Flow: {price_to_cash_flow(data["price"], data["cash_flow"]):.2f}', ln=2)






# Add page 2
pdf.add_page()

# At the top of the page add a multi-cell with text that will be wrapped and import the text from the growth txt file
pdf.set_xy(10, 10)
pdf.set_font('Arial', 'B', 11)
pdf.set_text_color(0, 0, 0)
pdf.cell(0, 5, f'Growth Overview', ln=2)
pdf.set_font('Arial', '', 10)
pdf.set_text_color(0, 0, 0)
with open('Growth_3.txt', 'r') as file:
    text = file.read()
pdf.multi_cell(110, 6, text, align='J')

# on right hand side of the page add a cell for the global product mix chart
pdf.set_xy(125, 10)
pdf.set_font('Arial', 'B', 10)
pdf.set_text_color(0, 0, 0)
pdf.cell(0, 5, f'Global Product Mix', ln=2)
with open('product_mix.png', 'rb') as img_file:
    pdf.image(img_file, 125, 17, 70, 60)

# Add a line under the cell
pdf.set_xy(125, 80)
pdf.set_line_width(0.5)
pdf.set_draw_color(0, 0, 0)
pdf.line(125, 80, 200, 80)

# Under the growth overview add a multi cell with text that will be wrapped and import the text from the market txt file
pdf.set_xy(10, 80)
pdf.set_font('Arial', 'B', 11)
pdf.set_text_color(0, 0, 0)
pdf.cell(0, 5, f'Market Overview', ln=2)
pdf.set_font('Arial', '', 10)
pdf.set_text_color(0, 0, 0)
with open('Market_5.txt', 'r') as file:
    text = file.read()
pdf.multi_cell(110, 6, text, align='J')

# Under the product mix chart add a cell for the sales breakdown chart
pdf.set_xy(125, 83)
pdf.set_font('Arial', 'B', 10)
pdf.set_text_color(0, 0, 0)
pdf.cell(0, 5, f'Sales Breakdown', ln=2)
with open('geo_sales.png', 'rb') as img_file:
    pdf.image(img_file, 125, 90, 70, 60)

# Under the market overview add a multi cell with text that will be wrapped and import the text from the sustainability txt file
pdf.set_xy(10, 167)
pdf.set_font('Arial', 'B', 11)
pdf.set_text_color(0, 0, 0)
pdf.cell(0, 5, f'Sustainability Targets', ln=2)
pdf.set_font('Arial', '', 10)
pdf.set_text_color(0, 0, 0)
with open('Sustain_4.txt', 'r') as file:
    text = file.read()
pdf.multi_cell(110, 6, text, align='J')

# Under the sustainability overview add a cell for the dcf extract 


# Next to the dcf extract add a cell for the valuation calculation 




# now write the file (closes the PDF)
pdf.output('aGroup 59 Analyst Report.pdf')

C:\Users\tomsc\AppData\Local\Temp\ipykernel_32956\1583665759.py:17: DeprecationWarning: Substituting font arial by core font helvetica - This is deprecated since v2.7.8, and will soon be removed
  pdf.set_font('Arial', 'B', 10)
C:\Users\tomsc\AppData\Local\Temp\ipykernel_32956\1583665759.py:21: DeprecationWarning: The parameter "ln" is deprecated since v2.5.2. Instead of ln=2 use new_x=XPos.LEFT, new_y=YPos.NEXT.
  pdf.cell(0, 5, f'FINM3422 Equity Research Report', ln=2)
C:\Users\tomsc\AppData\Local\Temp\ipykernel_32956\1583665759.py:23: DeprecationWarning: Substituting font arial by core font helvetica - This is deprecated since v2.7.8, and will soon be removed
  pdf.set_font('Arial', '', 10)
C:\Users\tomsc\AppData\Local\Temp\ipykernel_32956\1583665759.py:25: DeprecationWarning: The parameter "ln" is deprecated since v2.5.2. Instead of ln=1 use new_x=XPos.LMARGIN, new_y=YPos.NEXT.
  pdf.cell(0, 5, '30th April 2025', ln=1)
C:\Users\tomsc\AppData\Local\Temp\ipykernel_32956\1583665759.py